In [ ]:
# Import custom modules
import config
import data_processing as dp
import model_training as mt
import visualization as viz

# Standard imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Create output directories if they don't exist
import os
os.makedirs(config.OUTPUT_DIR, exist_ok=True)
os.makedirs(config.MODELS_DIR, exist_ok=True)
os.makedirs(config.PLOTS_DIR, exist_ok=True)

print("✓ All modules imported successfully")
print(f"✓ Project: {config.PROJECT_NAME}")
print(f"✓ Authors: {', '.join(config.AUTHORS)}")

In [ ]:
# Initialize Spark
spark = dp.initialize_spark("NYC_Payroll_ML_Analysis")

In [ ]:
# Execute complete data processing pipeline
df_clean = dp.full_data_pipeline(
    spark=spark,
    file_path=config.DATA_PATH,
    years=config.FISCAL_YEARS
)

In [ ]:
# Prepare features for salary prediction
df_model1 = dp.prepare_model1_features(df_clean)

# Convert to Pandas for XGBoost
df_model1_pd = df_model1.toPandas()
print(f"✓ Converted to Pandas: {len(df_model1_pd):,} rows")

In [ ]:
# Reduce cardinality of categorical variables
df_model1_pd, top_agencies, top_titles = dp.reduce_cardinality(
    df_model1_pd,
    top_n_agencies=config.TOP_N_AGENCIES,
    top_n_titles=config.TOP_N_TITLES
)

In [ ]:
# Encode features
X1, y1 = dp.encode_features(
    df_model1_pd,
    target_col='Base Salary',
    exclude_cols=[]
)

print(f"\nFeature Matrix Shape: {X1.shape}")
print(f"Target Variable Shape: {y1.shape}")
print(f"\nFirst 10 Features: {list(X1.columns[:10])}")

In [ ]:
# Split data
X1_train, X1_test, y1_train, y1_test = mt.split_data(X1, y1)

In [ ]:
# Train and evaluate complete pipeline for Model 1
model1, metrics1, importance1 = mt.train_and_evaluate_salary_model(
    X1_train, X1_test, y1_train, y1_test
)

In [ ]:
# Generate all visualizations for Model 1
viz.create_model1_visualizations(
    model=model1,
    X_test=X1_test,
    y_test=y1_test,
    importance_df=importance1,
    save_dir=config.PLOTS_DIR
)

In [ ]:
print("\n" + "="*80)
print("MODEL 1 SUMMARY: SALARY PREDICTION")
print("="*80)
print(f"\n🎯 Business Question Answered:")
print("   How do tenure, geography, and agency affect base salary?\n")

print(f"📊 Performance Metrics:")
for metric, value in metrics1.items():
    if 'r2' in metric:
        print(f"   {metric}: {value:.4f} ({value*100:.2f}%)")
    else:
        print(f"   {metric}: ${value:,.2f}")

print(f"\n🔝 Top 5 Features:")
for idx, row in importance1.head(5).iterrows():
    print(f"   {row['Feature'][:45]:45s} : {row['Importance']:.4f}")

print(f"\n Outputs Generated:")
print(f"   • Model saved: {config.XGBOOST_MODEL_PATH}")
print(f"   • Predictions: {config.OUTPUT_DIR}{config.SALARY_PREDICTIONS_FILE}")
print(f"   • Feature Importance: {config.OUTPUT_DIR}{config.SALARY_FEATURE_IMPORTANCE_FILE}")

In [ ]:
# Prepare features for overtime risk classification
df_model2 = dp.prepare_model2_features(df_clean)

# Convert to Pandas
df_model2_pd = df_model2.toPandas()
print(f"✓ Converted to Pandas: {len(df_model2_pd):,} rows")

In [ ]:
# Reduce cardinality (reuse top agencies and titles from Model 1)
df_model2_pd['Agency Name'] = df_model2_pd['Agency Name'].apply(
    lambda x: x if x in top_agencies else 'OTHER'
)
df_model2_pd['Title Description'] = df_model2_pd['Title Description'].apply(
    lambda x: x if x in top_titles else 'OTHER'
)

print("✓ Cardinality reduced using Model 1 categories")

In [ ]:
# Encode features (exclude OT-related columns to prevent data leakage)
X2, y2 = dp.encode_features(
    df_model2_pd,
    target_col='At_Risk',
    exclude_cols=['OT Hours', 'Total OT Paid', 'OT_Risk_Category']
)

print(f"\nFeature Matrix Shape: {X2.shape}")
print(f"Target Variable Shape: {y2.shape}")
print(f"\nClass Distribution:\n{y2.value_counts()}")

In [ ]:
# Split data with stratification for imbalanced classes
X2_train, X2_test, y2_train, y2_test = mt.split_data(X2, y2, stratify=y2)

In [ ]:
# Train and evaluate complete pipeline for Model 2
model2, metrics2, importance2 = mt.train_and_evaluate_risk_model(
    X2_train, X2_test, y2_train, y2_test
)

In [ ]:
# Generate all visualizations for Model 2
viz.create_model2_visualizations(
    model=model2,
    X_test=X2_test,
    y_test=y2_test,
    importance_df=importance2,
    save_dir=config.PLOTS_DIR
)

In [ ]:
print("\n" + "="*80)
print("MODEL 2 SUMMARY: OVERTIME RISK CLASSIFICATION")
print("="*80)
print(f"\n🎯 Business Question Answered:")
print("   Which employees are at high risk of overtime burnout?\n")

print(f"📊 Performance Metrics:")
for metric, value in metrics2.items():
    print(f"   {metric}: {value:.4f}")

print(f"\n🔝 Top 5 Risk Factors:")
for idx, row in importance2.head(5).iterrows():
    print(f"   {row['Feature'][:45]:45s} : {row['Importance']:.4f}")

print(f"\n💾 Outputs Generated:")
print(f"   • Model saved: {config.RF_MODEL_PATH}")
print(f"   • Predictions: {config.OUTPUT_DIR}{config.RISK_PREDICTIONS_FILE}")
print(f"   • Feature Importance: {config.OUTPUT_DIR}{config.RISK_FEATURE_IMPORTANCE_FILE}")

In [ ]:
print("\n" + "="*80)
print("FINAL BUSINESS INSIGHTS - ANSWERING KEY QUESTIONS")
print("="*80)

print("\n📊 QUESTION 1: How do tenure and geography affect salary?")
print("-" * 80)

# Calculate importance by category
tenure_imp = importance1[importance1['Feature'] == 'Years of Service']['Importance'].sum()
borough_imp = importance1[importance1['Feature'].str.contains('Borough', na=False)]['Importance'].sum()
agency_imp = importance1[importance1['Feature'].str.contains('Agency', na=False)]['Importance'].sum()

print(f"\n✓ TENURE Impact: {tenure_imp*100:.1f}%")
print(f"  → Years of service is {'THE STRONGEST' if tenure_imp > 0.2 else 'a SIGNIFICANT'} predictor")
print(f"  → Each additional year contributes to salary determination")

print(f"\n✓ GEOGRAPHY Impact: {borough_imp*100:.1f}%")
print(f"  → Borough location affects {borough_imp*100:.1f}% of salary variance")
print(f"  → {'Significant' if borough_imp > 0.05 else 'Moderate'} geographic salary differences exist")

print(f"\n✓ AGENCY Impact: {agency_imp*100:.1f}%")
print(f"  → Agency affiliation explains {agency_imp*100:.1f}% of variance")

print(f"\n💡 MODEL PERFORMANCE: R² = {metrics1['test_r2']:.4f}")
print(f"  → Can predict salary with average error of ${metrics1['test_mae']:,.0f}")
print(f"  → Model explains {metrics1['test_r2']*100:.1f}% of all salary variation")

In [ ]:
print("\n" + "="*80)
print("📊 QUESTION 2: Who is at overtime burnout risk?")
print("-" * 80)

print(f"\n✓ MODEL ACCURACY: {metrics2['test_accuracy']*100:.1f}%")
print(f"  → Correctly identifies overtime risk with {metrics2['test_accuracy']*100:.1f}% accuracy")

print(f"\n✓ RECALL: {metrics2['recall']*100:.1f}%")
print(f"  → Detects {metrics2['recall']*100:.1f}% of actual high-risk employees")
print(f"  → {'Good performance' if metrics2['recall'] >= config.MIN_ACCEPTABLE_RECALL else 'Warning: May miss some at-risk employees'}")

print(f"\n✓ PRECISION: {metrics2['precision']*100:.1f}%")
print(f"  → When flagged, {metrics2['precision']*100:.1f}% are truly at risk")
print(f"  → Minimizes false alarms for HR intervention")

# Top risk factors
print(f"\n✓ TOP RISK FACTORS:")
for idx, row in importance2.head(3).iterrows():
    feature = row['Feature']
    if 'Agency' in feature:
        print(f"  → {feature}: Certain agencies have higher OT dependency")
    elif 'Title' in feature:
        print(f"  → {feature}: Job role strongly predicts overtime risk")
    elif 'Salary' in feature:
        print(f"  → {feature}: Salary level correlates with OT patterns")
    else:
        print(f"  → {feature}: Contributes {row['Importance']*100:.1f}% to risk")

In [ ]:
print("\n" + "="*80)
print("🎯 ACTIONABLE RECOMMENDATIONS")
print("-" * 80)
print("\n1. SALARY EQUITY AUDIT")
print(f"   → Use Model 1 predictions to identify employees earning ${metrics1['test_mae']*2:,.0f}+ below expected")
print("   → Review geographic pay disparities across boroughs")
print("   → Investigate tenure-salary relationship for fair compensation")

print("\n2. OVERTIME RISK MANAGEMENT")
print(f"   → Deploy Model 2 quarterly to identify high-risk employees")
print(f"   → Focus on {importance2.head(1)['Feature'].values[0]} as primary intervention point")
print("   → Proactive wellness programs for flagged employees")

print("\n3. STRATEGIC WORKFORCE PLANNING")
print("   → Forecast salary budgets using Model 1 for headcount planning")
print("   → Redistribute workload in high-OT agencies identified by Model 2")
print("   → Target recruitment in understaffed departments")

print("\n" + "="*80)
print("PROJECT COMPLETE - ALL OUTPUTS SAVED")
print("="*80)